<a href="https://colab.research.google.com/github/CPTR295/ML1/blob/main/Titanic_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd
import numpy as np
import urllib.request

In [3]:
Titanic_path=os.path.join('datasets','titanic')
Download_path="https://raw.githubusercontent.com/CPTR295/ML1/main/datasets/titanic/"

def fetch_titanic_data(titanic_path=Titanic_path,download_path=Download_path):
  if not os.path.isdir(titanic_path):
    os.makedirs(titanic_path)
  for filename in ("train.csv","test.csv"):
    filepath = os.path.join(titanic_path,filename)
    if not os.path.isfile(filepath):
      print("Downloading",filename)
      urllib.request.urlretrieve(download_path+filename,filepath)

fetch_titanic_data()

In [4]:
def load_data(filename,path=Titanic_path):
  csv_path=os.path.join(path,filename)
  return pd.read_csv(csv_path)

In [5]:
train_data=load_data("train.csv")
test_data=load_data("test.csv")

In [6]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
test_data.head() #Test data doesnot have label , which is used for kaggle report

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
train_data=train_data.set_index("PassengerId") #Set passengerID to index
test_data=test_data.set_index("PassengerId")

In [9]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [11]:
#Lets ingnore Name,Ticket as these does not provide useful information , cabin as max rows are null

In [12]:
#For age lets use imputer
train_data[train_data["Sex"]=="female"]["Age"].median()

27.0

In [13]:
train_data.describe() #idea is use imputer on age and scale age and fare

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699113,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526507,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.416700,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [14]:
train_data["Survived"].value_counts()

,count
Survived,
0,549
1,342


In [15]:
train_data["Sex"].value_counts()

,count
Sex,
male,577
female,314


In [16]:
train_data2=train_data[["Survived","Pclass","Age","SibSp","Parch","Fare"]]

In [17]:
corr_matrix = train_data2.corr()

In [18]:
corr_matrix["Survived"]

,Survived
Survived,1.000000
Pclass,-0.338481
Age,-0.077221
SibSp,-0.035322
Parch,0.081629
Fare,0.257307


In [19]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

In [20]:
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ("onehotencoder",OneHotEncoder())
])

In [31]:
num_attribs = ["Age","SibSp","Parch","Fare"]
cat_attribs=["Embarked","Sex","Pclass"]

In [32]:
from sklearn.compose import ColumnTransformer

Full_pileline= ColumnTransformer([
    ("num",num_pipeline,num_attribs),
    ("cat",cat_pipeline,cat_attribs)
])

In [33]:
train_prepared=Full_pileline.fit_transform(train_data) #Return an NP Array

In [34]:
train_prepared

array([[-0.56573582,  0.43279337, -0.47367361, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.6638609 ,  0.43279337, -0.47367361, ...,  1.        ,
         0.        ,  0.        ],
       [-0.25833664, -0.4745452 , -0.47367361, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.10463705,  0.43279337,  2.00893337, ...,  0.        ,
         0.        ,  1.        ],
       [-0.25833664, -0.4745452 , -0.47367361, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.20276213, -0.4745452 , -0.47367361, ...,  0.        ,
         0.        ,  1.        ]])

In [35]:
y_train = train_data['Survived']

In [36]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier()
rf.fit(train_prepared,y_train)

RandomForestClassifier()

In [40]:
test_prepared=Full_pileline.transform(test_data)

In [41]:
y_preds = rf.predict(test_prepared)

In [44]:
from sklearn.model_selection import cross_val_score
rf_scores=cross_val_score(rf,train_prepared,y_train,cv=3)
rf_scores.mean()

np.float64(0.787878787878788)

In [46]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators':[3,10,30],'max_features':[2,4,6,8]}, #12 combinations
    {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]}, # 6 combinations
]

grid_search=GridSearchCV(rf,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(train_prepared,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [47]:
grid_search.best_estimator_


RandomForestClassifier(max_features=8, n_estimators=30)

In [48]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

ran_search_cv = RandomizedSearchCV(rf,param_distribs,n_iter=10,cv=5,scoring='neg_mean_squared_error', random_state=42)
ran_search_cv.fit(train_prepared,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x798a16176e70>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x798a1334c7d0>},
                   random_state=42, scoring='neg_mean_squared_error')

In [49]:
ran_search_cv.best_estimator_

RandomForestClassifier(max_features=5)

In [53]:
from sklearn.metrics import confusion_matrix
y2_pred=rf.predict(train_prepared)
cm=confusion_matrix(y_train,y2_pred)
cm

array([[542,   7],
       [ 11, 331]])

In [54]:
from sklearn.svm import SVC
svm_clf=SVC(gamma='auto')
svc_scores = cross_val_score(svm_clf,train_prepared,y_train,cv=7)
svc_scores.mean()

np.float64(0.8283007592800901)

In [66]:
rf1=RandomForestClassifier(max_features=5,random_state=42)
rf1_scores=cross_val_score(rf1,train_prepared,y_train,cv=3)
rf1_scores.mean()

np.float64(0.7946127946127945)

In [67]:

rf2=RandomForestClassifier(max_features=8, n_estimators=30,random_state=42)
rf2_scores=cross_val_score(rf2,train_prepared,y_train,cv=3)
rf2_scores.mean()

np.float64(0.8047138047138048)